# Vanilla Option

In [7]:
import QuantLib as ql
from src.QlMarket import QlMarket
from src.QlStockPricer import QlStockPricer
from src.QlVanillaOption import QlVanillaOption

## 初始化市场

In [8]:
start_date = ql.Date(1, 1, 2023)
qlMarket = QlMarket(init_date=start_date)
start_date = qlMarket.init_date
start_date

Date(3,1,2023)

## 初始化一个股票

In [9]:
stock_1 = QlStockPricer(QlMarket=qlMarket, init_stock_price=100.0)
stock_1.using_Black_Scholes_model(sigma=0.2)

Next step, you need set model type
set new model: Black Scholes model with sigma: 0.2


In [10]:
end_date = qlMarket.cal_date_advance(start_date,1, 'months')
end_date

Date(3,2,2023)

## 初始化期权

In [11]:
payoff = ql.PlainVanillaPayoff(ql.Option.Call, 100.0)
exercise = ql.EuropeanExercise(end_date)

In [12]:
option = QlVanillaOption(
    payoff,
    exercise,
    QlStockPricer = stock_1,
    engine = ql.AnalyticEuropeanEngine
)

In [7]:
option.stock_price.value()

100.0

In [8]:
option.today

Date(3,1,2023)

In [9]:
option.NPV()

2.446616201530256

## 初始化期权（方式二）

In [ ]:
option_0 = QlVanillaOption.init_from_price_and_date(
    strike_price=100.0,
    end_date=end_date,
    QlStockPricer=stock_1,
    engine = ql.AnalyticEuropeanEngine
)
option_0.NPV()

## greeks

In [10]:
option.greeks()

{'delta': 0.5392725928973825,
 'gamma': 0.07046187392115945,
 'theta': -16.66640693864226,
 'vega': 11.184424431930074,
 'rho': 4.085765324460952}

### Vanilla Option 定价引擎

#### 1. **解析法引擎**
- **`AnalyticEuropeanEngine`**  
  - **用途**：欧式期权的解析定价（Black-Scholes 公式）  
  - **输入**：`GeneralizedBlackScholesProcess`（包含标的、无风险利率、股息率、波动率）  
  - **特点**：快速精确，仅支持欧式期权。

#### 2. **蒙特卡洛引擎**
- **`MCEuropeanEngine`**  
  - **用途**：欧式期权的蒙特卡洛模拟定价  
  - **输入**：`GeneralizedBlackScholesProcess` + 随机数生成器（如伪随机/低差异序列）  
  - **参数**：时间步数、路径数量、容差等。  
  - **特点**：支持复杂路径依赖期权，计算较慢但灵活。

- **`MCAmericanEngine`**  
  - **用途**：美式期权的蒙特卡洛模拟定价（LSM算法）  
  - **输入**：类似 `MCEuropeanEngine`，但需校准多项式。  
  - **特点**：支持提前行权，计算成本高。

#### 3. **有限差分引擎**
- **`FdBlackScholesVanillaEngine`**  
  - **用途**：欧式/美式期权的有限差分法定价  
  - **输入**：`GeneralizedBlackScholesProcess` + 网格参数（时间/空间步数）  
  - **特点**：高效处理美式期权，支持局部波动率。

#### 4. **Heston模型引擎**
- **`AnalyticHestonEngine`**  
  - **用途**：欧式期权的Heston模型解析解  
  - **输入**：`HestonProcess`（包含随机波动率参数）  
  - **特点**：考虑波动率微笑，计算速度快。

- **`FdHestonVanillaEngine`**  
  - **用途**：Heston模型下的有限差分定价  
  - **输入**：`HestonModel` + 三维网格（标的/波动率/时间）  
  - **特点**：支持美式期权和复杂边界条件。

#### 5. **其他扩展引擎**
- **`AnalyticPTDHestonEngine`**  
  - **用途**：分段时变Heston模型的解析定价。  
- **`MCEuropeanHestonEngine`**  
  - **用途**：Heston模型下的蒙特卡洛模拟。

---

### 选择建议
- **欧式期权**：优先用 `AnalyticEuropeanEngine` 或 `AnalyticHestonEngine`。  
- **美式期权**：使用 `FdBlackScholesVanillaEngine` 或 `MCAmericanEngine`。  
- **波动率微笑**：Heston系列引擎（解析或有限差分）。  
- **路径依赖**：蒙特卡洛引擎（如亚式、障碍期权）。

--- 

可根据实际需求进一步补充参数说明或代码示例。

In [11]:
steps = qlMarket.day_counter.dayCount(qlMarket.today, end_date)
steps

20

In [12]:
option2 = QlVanillaOption(
    payoff,
    exercise,
    stock_1,
    ql.MCEuropeanEngine,
    traits = "pseudorandom", # could use "lowdiscrepancy"
    timeSteps = steps,
    requiredSamples = 10000,
)

In [14]:
option.NPV()

2.446616201530256

In [17]:
option2.NPV()

2.4857584909268216

## Mont Carlo do not have default greeks

In [18]:
option2.delta()

RuntimeError: delta not provided

# heston

In [19]:
stock_1.using_Heston_model(v0=0.005, kappa=0.8, theta=0.008, rho=0.2, sigma=0.1)

set new model: Heston model with yield rate: v0: 0.005, kappa: 0.8, theta: 0.008, rho: 0.2, sigma: 0.1, None


In [20]:
option3 = QlVanillaOption(
    payoff,
    exercise,
    QlStockPricer = stock_1,
    engine = ql.AnalyticHestonEngine
)

In [21]:
option3.NPV()

1.0065728067960784

In [22]:
option3 = QlVanillaOption(
    payoff,
    exercise,
    QlStockPricer = stock_1,
    engine = ql.MCEuropeanHestonEngine,
    traits = "pseudorandom", # could use "lowdiscrepancy"
    timeSteps = steps,
    requiredSamples = 10000,
)

TypeError: in method 'new_MCPREuropeanHestonEngine', argument 1 of type 'ext::shared_ptr< HestonProcess > const &'